# Custom RAG System (Gemini Flash)

## Phase 1: Environment & Project Setup

This phase ensures:
- Clean project structure
- Isolated Python environment
- Secure secrets management
- Reproducible execution

In [1]:
import sys
print("Python executable:", sys.executable)
print("Python version:", sys.version)

Python executable: c:\Users\GCV\.gemini\antigravity\scratch\notebooklm-rag-comparison\rag-venv\Scripts\python.exe
Python version: 3.14.2 (tags/v3.14.2:df79316, Dec  5 2025, 17:18:21) [MSC v.1944 64 bit (AMD64)]


In [2]:
import platform
print("OS:", platform.system())

OS: Windows


In [3]:
import os
from dotenv import load_dotenv

load_dotenv()

print("GOOGLE_API_KEY loaded:", bool(os.getenv("GOOGLE_API_KEY")))
print("PINECONE_API_KEY loaded:", bool(os.getenv("PINECONE_API_KEY")))

GOOGLE_API_KEY loaded: True
PINECONE_API_KEY loaded: True


In [4]:
print("Phase 1 setup complete and verified.")

Phase 1 setup complete and verified.


# Phase 2: Document Ingestion & Text Extraction

This phase:
- Loads PDFs from disk
- Extracts page-level text
- Preserves metadata (source + page)
- Performs validation and sanity checks

In [5]:
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader

c:\Users\GCV\.gemini\antigravity\scratch\notebooklm-rag-comparison\rag-venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [6]:
PDF_DIR = Path("data/pdfs")
assert PDF_DIR.exists(), "data/pdfs folder does not exist"

pdf_files = sorted(PDF_DIR.glob("*.pdf"))

print(f"Found {len(pdf_files)} PDF files:")
for f in pdf_files:
    print("-", f.name)

assert len(pdf_files) > 0, "No PDFs found. Add files to data/pdfs/"

Found 7 PDF files:
- lech201.pdf
- lech202.pdf
- lech203.pdf
- lech204.pdf
- lech205.pdf
- lech2an.pdf
- lech2ps.pdf


In [7]:
documents = []

for pdf_path in pdf_files:
    print(f"\nLoading: {pdf_path.name}")
    loader = PyPDFLoader(str(pdf_path))
    pages = loader.load()

    print(f"  Pages extracted: {len(pages)}")
    documents.extend(pages)

print(f"\nTotal pages loaded from all PDFs: {len(documents)}")


Loading: lech201.pdf
  Pages extracted: 34

Loading: lech202.pdf
  Pages extracted: 34

Loading: lech203.pdf
  Pages extracted: 32

Loading: lech204.pdf
  Pages extracted: 22

Loading: lech205.pdf
  Pages extracted: 22

Loading: lech2an.pdf
  Pages extracted: 6

Loading: lech2ps.pdf
  Pages extracted: 14

Total pages loaded from all PDFs: 164


In [8]:
sample = documents[0]
print("Metadata:", sample.metadata)
print("\n--- Content Preview (first 1000 chars) ---\n")
print(sample.page_content[:1000])

Metadata: {'producer': 'PDF Printer / www.bullzip.com / FG / Freeware Edition (max 10 users)', 'creator': 'Bullzip PDF Printer (12.2.0.2905)', 'creationdate': '2022-10-14T16:03:07-07:00', 'author': '205DTP3', 'moddate': '2025-04-08T10:06:41+05:30', 'title': 'D:\\TEXTBOOKS\\RATIONALISED 20222-23\\Neha\\12086 — Chemistry Part_II\\1 Source Files\\12086 — Chemistry Part_II\\Unit_6\\Unit_6.pmd', 'source': 'data\\pdfs\\lech201.pdf', 'total_pages': 34, 'page': 0, 'page_label': '1'}

--- Content Preview (first 1000 chars) ---

The replacement of hydrogen atom(s) in a n aliphatic
or aromatic  hydrocarbon by halogen atom(s) results
in the formation of alkyl halide (haloalkane) and aryl
halide (haloarene), respectively. Haloalkanes contain
halogen atom(s) attached to the sp3 hybridised carbon
atom of an alkyl group whereas haloarenes contain
halogen atom(s) attached to sp2 hybridised carbon
atom(s) of an aryl group. Many halogen containing
organic compounds occur in nature and some of
these are c

In [9]:
empty_pages = [i for i, d in enumerate(documents) if len(d.page_content.strip()) < 50]

print("Total pages:", len(documents))
print("Empty or near-empty pages:", len(empty_pages))

if len(empty_pages) > 0:
    print("Indices of empty pages:", empty_pages[:10])

Total pages: 164
Empty or near-empty pages: 4
Indices of empty pages: [148, 149, 150, 155]


In [10]:
total_chars = sum(len(d.page_content) for d in documents)
avg_chars = total_chars / len(documents)

print("Total characters:", total_chars)
print("Average characters per page:", int(avg_chars))

Total characters: 270106
Average characters per page: 1646


In [11]:
print("Phase 2 complete: Documents ingested and validated.")

Phase 2 complete: Documents ingested and validated.


# Phase 3: Text Normalization & Chunking

This phase:
- Normalizes raw extracted text
- Removes formatting noise
- Prepares semantically meaningful chunks
- Preserves source metadata for traceability

In [12]:
import re
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
def normalize_text(text: str) -> str:
    # Collapse multiple newlines
    text = re.sub(r"\n+", "\n", text)

    # Collapse excessive whitespace
    text = re.sub(r"\s+", " ", text)

    # Strip edges
    text = text.strip()

    return text

In [14]:
for doc in documents:
    doc.page_content = normalize_text(doc.page_content)

print("Text normalization complete.")

Text normalization complete.


In [15]:
print("=== NORMALIZED TEXT SAMPLE ===\n")
print(documents[0].page_content[:1000])

=== NORMALIZED TEXT SAMPLE ===

The replacement of hydrogen atom(s) in a n aliphatic or aromatic hydrocarbon by halogen atom(s) results in the formation of alkyl halide (haloalkane) and aryl halide (haloarene), respectively. Haloalkanes contain halogen atom(s) attached to the sp3 hybridised carbon atom of an alkyl group whereas haloarenes contain halogen atom(s) attached to sp2 hybridised carbon atom(s) of an aryl group. Many halogen containing organic compounds occur in nature and some of these are clinically useful. These classes of compounds find wide applications in industry as well as in day- to-day life. They are used as solvents for relatively non-polar compounds and as starting materials for the synthesis of wide range of organic compounds. Chlorine containing antibiotic, chloramphenicol, produced by microorganisms is very effective for the treatment of typhoid fever . Our body pr oduces iodine containing hormone, thyroxine, the deficiency of which causes a disease called goite

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=150,
    separators=["\n\n", "\n", ". ", " ", ""]
)

In [17]:
chunks = text_splitter.split_documents(documents)

print("Total pages:", len(documents))
print("Total chunks created:", len(chunks))

Total pages: 164
Total chunks created: 384


In [18]:
sample_chunk = chunks[0]

print("Chunk metadata:", sample_chunk.metadata)
print("\n--- Chunk preview ---\n")
print(sample_chunk.page_content[:1000])

Chunk metadata: {'producer': 'PDF Printer / www.bullzip.com / FG / Freeware Edition (max 10 users)', 'creator': 'Bullzip PDF Printer (12.2.0.2905)', 'creationdate': '2022-10-14T16:03:07-07:00', 'author': '205DTP3', 'moddate': '2025-04-08T10:06:41+05:30', 'title': 'D:\\TEXTBOOKS\\RATIONALISED 20222-23\\Neha\\12086 — Chemistry Part_II\\1 Source Files\\12086 — Chemistry Part_II\\Unit_6\\Unit_6.pmd', 'source': 'data\\pdfs\\lech201.pdf', 'total_pages': 34, 'page': 0, 'page_label': '1'}

--- Chunk preview ---

The replacement of hydrogen atom(s) in a n aliphatic or aromatic hydrocarbon by halogen atom(s) results in the formation of alkyl halide (haloalkane) and aryl halide (haloarene), respectively. Haloalkanes contain halogen atom(s) attached to the sp3 hybridised carbon atom of an alkyl group whereas haloarenes contain halogen atom(s) attached to sp2 hybridised carbon atom(s) of an aryl group. Many halogen containing organic compounds occur in nature and some of these are clinically useful

In [19]:
sizes = [len(c.page_content) for c in chunks]

print("Min chunk size:", min(sizes))
print("Max chunk size:", max(sizes))
print("Avg chunk size:", sum(sizes) // len(sizes))

Min chunk size: 15
Max chunk size: 1000
Avg chunk size: 741


In [20]:
print("Phase 3 complete: Text normalized and chunked successfully.")

Phase 3 complete: Text normalized and chunked successfully.


# Phase 4: Embedding Generation (Sentence Transformers)

This phase:
- Initializes a HuggingFace embedding model
- Converts text chunks into vector embeddings
- Verifies embedding shape and consistency

In [21]:
from sentence_transformers import SentenceTransformer
import numpy as np

In [22]:
EMBEDDING_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"

print("Loading embedding model:", EMBEDDING_MODEL_NAME)
embedder = SentenceTransformer(EMBEDDING_MODEL_NAME)

print("Model loaded.")

Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded.


In [23]:
texts = [chunk.page_content for chunk in chunks]
print("Total chunks to embed:", len(texts))

Total chunks to embed: 384


In [24]:
embeddings = embedder.encode(
    texts,
    batch_size=32,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
)

print("Embeddings generated.")

Batches:   0%|          | 0/12 [00:00<?, ?it/s]

Embeddings generated.


In [25]:
print("Embeddings array shape:", embeddings.shape)
print("Single embedding dimension:", embeddings.shape[1])

Embeddings array shape: (384, 384)
Single embedding dimension: 384


In [26]:
print("Sample embedding (first 10 values):")
print(embeddings[0][:10])
print("Vector norm:", np.linalg.norm(embeddings[0]))

Sample embedding (first 10 values):
[ 0.00261767 -0.03758737 -0.08557532 -0.03558457  0.12289263  0.07794212
  0.02970156  0.0951428   0.06869452 -0.02832721]
Vector norm: 1.0


In [27]:
assert embeddings.shape[0] == len(chunks), "Mismatch between chunks and embeddings!"
assert embeddings.shape[1] == 384, "Unexpected embedding dimension!"

print("Embedding consistency checks passed.")

Embedding consistency checks passed.


In [28]:
print("Phase 4 complete: Embeddings generated and validated.")

Phase 4 complete: Embeddings generated and validated.


# Phase 5: Vector Database (Pinecone Serverless)

This phase:
- Initializes Pinecone Serverless client
- Creates a new serverless index (if needed)
- Batches and upserts vectors with metadata

In [29]:
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv

load_dotenv()

True

In [30]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
assert PINECONE_API_KEY is not None, "Missing PINECONE_API_KEY"

pc = Pinecone(api_key=PINECONE_API_KEY)
print("Pinecone client initialized.")

Pinecone client initialized.


In [31]:
INDEX_NAME = "notebooklm-rag-antigravity"

existing_indexes = [i["name"] for i in pc.list_indexes()]

if INDEX_NAME not in existing_indexes:
    print("Creating new serverless index:", INDEX_NAME)
    pc.create_index(
        name=INDEX_NAME,
        dimension=embeddings.shape[1],
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
else:
    print("Using existing index:", INDEX_NAME)

Using existing index: notebooklm-rag-antigravity


In [32]:
index = pc.Index(INDEX_NAME)

In [33]:
vectors = []

for i, (chunk, vector) in enumerate(zip(chunks, embeddings)):
    vectors.append({
        "id": f"chunk-{i}",
        "values": vector.tolist(),
        "metadata": {
            "source": chunk.metadata.get("source", ""),
            "page": chunk.metadata.get("page", ""),
            "text": chunk.page_content[:1000]  # metadata size safety
        }
    })

print("Prepared vectors:", len(vectors))

Prepared vectors: 384


In [34]:
BATCH_SIZE = 100

for i in range(0, len(vectors), BATCH_SIZE):
    batch = vectors[i:i+BATCH_SIZE]
    index.upsert(vectors=batch)
    print(f"Uploaded {min(i + BATCH_SIZE, len(vectors))} / {len(vectors)} vectors")

Uploaded 100 / 384 vectors
Uploaded 200 / 384 vectors
Uploaded 300 / 384 vectors
Uploaded 384 / 384 vectors


In [35]:
stats = index.describe_index_stats()
print("Index stats:", stats)

Index stats: {'_response_info': {'raw_headers': {'connection': 'keep-alive',
                                    'content-length': '185',
                                    'content-type': 'application/json',
                                    'date': 'Wed, 28 Jan 2026 09:12:04 GMT',
                                    'grpc-status': '0',
                                    'server': 'envoy',
                                    'x-envoy-upstream-service-time': '42',
                                    'x-pinecone-request-id': '2147333110094194392',
                                    'x-pinecone-request-latency-ms': '41',
                                    'x-pinecone-response-duration-ms': '43'}},
 'dimension': 384,
 'index_fullness': 0.0,
 'memoryFullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'__default__': {'vector_count': 384}},
 'storageFullness': 0.0,
 'total_vector_count': 384,
 'vector_type': 'dense'}


In [36]:
print("Phase 5 complete: Vectors successfully stored in Pinecone Serverless.")

Phase 5 complete: Vectors successfully stored in Pinecone Serverless.


# Phase 6: Semantic Retrieval Layer

This phase:
- Encodes user queries
- Performs similarity search in Pinecone
- Retrieves top-k relevant chunks
- Allows inspection of retrieved context

In [37]:
import numpy as np

In [38]:
def embed_query(query: str) -> np.ndarray:
    vec = embedder.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    return vec[0]

In [39]:
def retrieve_top_k(query: str, k: int = 5):
    query_vec = embed_query(query)

    result = index.query(
        vector=query_vec.tolist(),
        top_k=k,
        include_metadata=True
    )

    return result["matches"]

In [40]:
TEST_QUERY = "What is the main topic discussed in these documents?"

results = retrieve_top_k(TEST_QUERY, k=5)

print("Retrieved chunks:", len(results))

Retrieved chunks: 5


In [41]:
for i, match in enumerate(results, 1):
    print(f"\n--- Result {i} ---")
    print("Score:", match["score"])
    print("Source:", match["metadata"].get("source"))
    print("Page:", match["metadata"].get("page"))
    print("\nText Preview:\n")
    print(match["metadata"].get("text")[:1000])


--- Result 1 ---
Score: 0.451507598
Source: data\pdfs\lech2an.pdf
Page: 5

Text Preview:

Notes Reprint 2025-26

--- Result 2 ---
Score: 0.451507598
Source: data\pdfs\lech2an.pdf
Page: 4

Text Preview:

Notes Reprint 2025-26

--- Result 3 ---
Score: 0.398946285
Source: data\pdfs\lech2ps.pdf
Page: 7

Text Preview:

. A team of experts constituted by the NCERT has developed the manuscript of the book. It gives me great pleasure to acknowledge the valuable contribution of all the members of this team. I also acknowledge the valuable and relentless contribution of the editors in bringing the book to the present shape. I also acknowledge with thanks the dedicated efforts and valuable contribution of Professor Brahm Parkash, who not only coordinated the entire programme but also actively involved in writing and editing of this book. Thanks are also due to the participating teachers and subject experts of the review workshop for their contribution, which has helped us to make the book learne

In [42]:
assert len(results) > 0, "No retrieval results returned!"
print("Retrieval sanity check passed.")

Retrieval sanity check passed.


In [43]:
print("Phase 6 complete: Semantic retrieval layer operational.")

Phase 6 complete: Semantic retrieval layer operational.


# Phase 7: RAG Answer Generation (Gemini Flash)

This phase:
- Integrates Gemini Flash as the LLM
- Combines semantic retrieval + generation
- Uses strict grounding to prevent hallucination
- Produces user-facing answers

In [44]:
import os
import google.generativeai as genai
from dotenv import load_dotenv

load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
assert GOOGLE_API_KEY is not None, "Missing GOOGLE_API_KEY"

genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel("gemini-flash-latest")
print("Gemini Flash initialized.")

Gemini Flash initialized.


C:\Users\GCV\AppData\Local\Temp\ipykernel_22736\182369644.py:2: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  import google.generativeai as genai


In [45]:
def build_context(retrieved_matches, max_chars=6000):
    context_blocks = []
    total_chars = 0

    for match in retrieved_matches:
        text = match["metadata"].get("text", "")
        source = match["metadata"].get("source", "")
        page = match["metadata"].get("page", "")

        block = f"[Source: {source}, Page: {page}]\n{text}\n"
        
        if total_chars + len(block) > max_chars:
            break

        context_blocks.append(block)
        total_chars += len(block)

    return "\n---\n".join(context_blocks)

In [46]:
def build_rag_prompt(context: str, question: str) -> str:
    prompt = f"""
You are a factual assistant.

Answer the question strictly using ONLY the context below.
If the answer is not present in the context, say:
"I don't have enough information in the provided documents."

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""
    return prompt.strip()

In [47]:
def rag_answer(query: str, k: int = 5):
    # Step 1: Retrieve relevant chunks
    retrieved = retrieve_top_k(query, k=k)

    # Step 2: Build context window
    context = build_context(retrieved)

    # Step 3: Build grounded prompt
    prompt = build_rag_prompt(context, query)

    # Step 4: Generate answer using Gemini
    response = model.generate_content(prompt)

    return {
        "query": query,
        "answer": response.text,
        "context": context,
        "retrieved": retrieved
    }

In [48]:
TEST_QUESTION = "Summarize the main topics discussed in these documents."

result = rag_answer(TEST_QUESTION, k=5)

print("QUESTION:\n", result["query"])
print("\nANSWER:\n", result["answer"])

QUESTION:
 Summarize the main topics discussed in these documents.

ANSWER:
 The documents primarily discuss the development and structure of a book or textbook manuscript, along with acknowledgments and publication details.

Key topics include:

1.  **Book Development and Structure:** A team of experts constituted by the NCERT developed the book's manuscript. The book's content features structural formulae of chemical compounds showing functional/coordinating groups in different colours, examples with solutions, intext questions, and end of Unit exercises designed to provoke thinking. It also incorporates non-evaluative material, such as biographical sketches of scientists and additional information, given in boxes with a deep yellow coloured bar. The aim is to make the book learner friendly and stimulate readers' interest in the subject.
2.  **Acknowledgments:** Valuable contributions were acknowledged from team members, editors, Professor Brahm Parkash (who coordinated the programme

In [49]:
print("\n=== CONTEXT USED FOR ANSWER ===\n")
print(result["context"])


=== CONTEXT USED FOR ANSWER ===

[Source: data\pdfs\lech2ps.pdf, Page: 7]
. A team of experts constituted by the NCERT has developed the manuscript of the book. It gives me great pleasure to acknowledge the valuable contribution of all the members of this team. I also acknowledge the valuable and relentless contribution of the editors in bringing the book to the present shape. I also acknowledge with thanks the dedicated efforts and valuable contribution of Professor Brahm Parkash, who not only coordinated the entire programme but also actively involved in writing and editing of this book. Thanks are also due to the participating teachers and subject experts of the review workshop for their contribution, which has helped us to make the book learner friendly. Also, I thank the technical and administrative staf f of the NCER T for their support in the entire process. The team of this textbook development programme hopes that the book stimulates its readers and makes them feel the excite

In [ ]:
print("\n🤖 Interactive Q&A Mode")
print("Type 'exit' to quit\n")

while True:
    user_question = input("\n❓ Your question: ")
    
    if user_question.lower() == "exit":
        break
    
    result = rag_answer(user_question, k=5)
    print(f"\n✅ ANSWER:\n{result['answer']}\n")


🤖 Interactive Q&A Mode
Type 'exit' to quit


✅ ANSWER:
I don't have enough information in the provided documents.


✅ ANSWER:
I don't have enough information in the provided documents.


✅ ANSWER:
I don't have enough information in the provided documents.


✅ ANSWER:
I don't have enough information in the provided documents.



In [ ]:
print("Phase 7 complete: End-to-end RAG system operational with Gemini Flash.")

Phase 7 complete: End-to-end RAG system operational with Gemini Flash.


# Phase 8: Evaluation & NotebookLM Comparison

This phase compares:
- NotebookLM output
- Custom RAG (Gemini Flash) output

Using:
- Same PDFs
- Same query

In [ ]:
NOTEBOOKLM_ANSWER = """
PASTE YOUR NOTEBOOKLM ANSWER HERE
"""

In [ ]:
EVAL_QUESTION = "Summarize the main contributions and limitations discussed across these documents."

rag_result = rag_answer(EVAL_QUESTION, k=5)

RAG_ANSWER = rag_result["answer"]

print("RAG ANSWER:\n")
print(RAG_ANSWER)

KeyboardInterrupt: 

In [ ]:
print("\n" + "="*80)
print("NOTEBOOKLM ANSWER:\n")
print(NOTEBOOKLM_ANSWER)

print("\n" + "="*80)
print("CUSTOM RAG ANSWER:\n")
print(RAG_ANSWER)

## Comparison Observations

### Similarities
- 
- 
 
### Differences
- 
- 

### Missing Points
- 

### Extra / Hallucinated Points
- 

### Verdict
- Parity: High / Medium / Low
- Reason: 